In [1]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import glob
import os
from os import *
import re
import imutils
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import collections
from ipynb.fs.full.Track_Functions import *

In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]


In [3]:
def get_time_point(path, time_point_1):   # time_point in form of 'Take #1 ' (remember the space between #1 ') or 'Take #32'

    image_time_point_1 = []
    

    for dirname, dirnames,filenames in os.walk(path):



        for filename in sorted(filenames, key = natural_keys):



            if time_point_1 in filename:
                
                im1 = cv2.imread(os.path.join(dirname, filename))
                if np.count_nonzero(im1 == 255) < 177770000:
                    image_time_point_1.append(im1)
                
            
            

    return image_time_point_1

In [4]:
def find_and_cut(path_img,time_point_1,save_path):

    
    img_list_1 = get_time_point(path_img,time_point_1)
    
    list_img = []
    list_img.append(img_list_1)
    
    
    wells = [time_point_1] 
    
    crop_count = []

    for list_ix, _ in enumerate(list_img):

        t = []
        t = list_img[list_ix]


        dict_cord = {}
        cor = []
        x = []
        y = []
        for i in tqdm.tqdm(t):
            cor_0,x0,y0 = return_cord(i)
            cor.append(cor_0)
            x.append(x0)
            y.append(y0)

            data={'X':x, 'Y':y, 'Cor':cor}
            data = pd.DataFrame(data)


        dict_cord = data
        coords = dict_cord.values.tolist()

        id_cell_pr = get_id_prime(coords[0][0],coords[0][1])


        rows_1_2, cols_1_2, list_id_index_1_2 ,d_1_2 = eucl_dist_matrix_t1(coords[0][2], coords[1][2])

        try:
            index, duplicate_columns = find_duplicates_col_t1(rows_1_2, cols_1_2, list_id_index_1_2, d_1_2)

            x_1, y_1, new_cor_1, id_cell_new = remove_duplicates_t1(coords[1][0],coords[1][1], index, id_cell_pr)

            new_coor = {'Cor_1':new_cor_1}

            new_id = {'ID_1':id_cell_new}

            for ix in range(len(t)):

                row, col, list_id_index1 ,d1 = eucl_dist_matrix_t1(list(new_coor.values())[ix], coords[2+ix][2])

                row1, col1 = zip(*list_id_index1)


                # Index duplicates cols


                duplicates2 = []
                for item in col1:
                    if col1.count(item) > 1:
                        duplicates2.append(item)
                dup_set2 = set(duplicates2)


                # Index unused cols

                r = len(list(new_coor.values())[ix])
                unused_col = list(range(0, r))
                unused = set(col1)
                idx_unused_col = [x for x in unused_col if x not in unused]


                n_cord_new, id_cell_new = final_fun(dup_set2, idx_unused_col, d1, list(new_id.values())[ix], list_id_index1, coords[2+ix][2], list(new_coor.values())[ix])

                new_coor['Cor_{}'.format(2+ix)] = n_cord_new
                new_id['ID_{}'.format(2+ix)] = id_cell_new


        except IndexError:
            pass


        del t[0]

        coor = []
        id_c = []

        for k, v in new_coor.items():
            coor.append(v)

        for p, c in new_id.items():
            id_c.append(c)

        for idx , _ in enumerate(t):

            img = t[idx]

            crop_cell_1 = []

            x = []
            y = []

            for ix, i in enumerate(coor[idx]):
                x.append(coor[idx][ix][0])      
                y.append(coor[idx][ix][1])

            #Image is going to be 90 x 90
            w = 45
            h = 45

            try_id = []

            for ix, i in enumerate(x):

                s = id_c[idx][ix]


                crop = img[(y[ix]-h):(y[ix]+h), (x[ix]-w):(x[ix]+w)]

                if crop.size == 24300: 

                    crop_cell_1.append(crop) 
                    try_id.append(s)

            for ls, i in enumerate(crop_cell_1):

                im = Image.fromarray(i)

                im.save(save_path.format(via, wells[list_ix],try_id[ls], int(idx+1)))  
                
            crop_count.append(len(crop_cell_1))
            
    return sum(crop_count)


In [10]:
p = '/home/jovyan/DATA_MASTER_PROJECT/IMG_A549_high_con/full_size_DMSO/'


In [11]:
name_well = []
for di,dr,f in os.walk(p):
    for i in f:
        name_well.append(i.split(' ')[1].split(' ')[0])

In [12]:
wells = list(set(name_well))
wells

['G10-3',
 'E10-5',
 'G11-1',
 'E6-2',
 'G10-1',
 'C11-3',
 'E10-4',
 'G10-4',
 'G11-3',
 'G4-3',
 'E6-4',
 'E10-1',
 'G11-2',
 'B9-2',
 'C6-2',
 'D3-1',
 'D3-4',
 'C11-4',
 'C11-2',
 'D3-5',
 'G4-2',
 'G11-4',
 'G4-5',
 'C11-1',
 'G4-4',
 'B9-4',
 'G11-5',
 'B9-3',
 'G10-2',
 'G4-1',
 'D3-2',
 'E10-3',
 'C11-5',
 'E10-2',
 'C6-3',
 'B9-5',
 'C6-1',
 'C6-5',
 'B9-1',
 'E6-5',
 'G10-5',
 'C6-4',
 'E6-3',
 'E6-1',
 'D3-3']

In [19]:
we = wells
del we[15]

In [20]:
via = '/home/jovyan/DATA_MASTER_PROJECT/IMG_A549_high_con/DMSO_cropped/'
directory = os.mkdir(via) 

In [21]:
# REMEMBER TO CHACHE ADR AND HRH

final_count = []

for i in wells:

    count = find_and_cut(p , i , '{0}/dmso_{1}_id_{2}_time_{3}.tiff')

    final_count.append(count)

100%|██████████| 34/34 [00:21<00:00,  1.60it/s]


In [ ]:


# To see if the cells that have the same id are the actual same cell in time 

img = []
for i in glob.glob(via + '/' + '*.tiff'):
    if 'C8-3_id_2_' in i:
        im = cv2.imread(i)
        img.append(im)
        
len(img)

In [ ]:
fig = plt.figure(figsize=(20, 20))


for i in range(30):
    ax = fig.add_subplot(3,10,i+1),plt.imshow(img[i],'gray')
    #plt.title(titles[i], color='white')
    plt.xticks([]),plt.yticks([])
plt.show()